In [2]:
from pickle import GLOBAL
import pinocchio as pin
import numpy as np
import time
import math
import sys
from numpy.linalg import norm, inv, pinv, det, matrix_rank
import pandas as pd

In [3]:
pinocchio_model_dir = '/home/unknown/Documents/Posdoc/ModelsCads/urdf' 
mesh_dir = pinocchio_model_dir + '/kinova_description/meshes/'
urdf_model_path =pinocchio_model_dir + '/j2s6s200_simple.urdf'
model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_model_path, mesh_dir)

In [4]:
frameIndex = model.getFrameId('j2s6s200_end_effector')
jointIndex = model.frames[frameIndex].parent
# Create data required by the algorithms
data     = model.createData()

In [17]:
def run_robot(): 
    # for runing the simulation
    t = 0
    dt = 1e-1

    tau_q = 0.375
    tau_x = 0.1
    
    md1 =15; md2 = 15;  md3 = 15
    mds = np.array([md1, md2, md3])
    Md = np.diag(mds) # Mass Matrix
    Md_inv = np.linalg.pinv(Md)
    
    # for graphs
    E1_list = []
    manu_list = []
    q_list = [] #q
    pose_list = []
    manipaxes_list = []
    manipaxesinv_list = []
    a = np.arange(47,180,1)
    b = np.arange(19,180,1)
    c = np.arange(65,295,1)

    ## for runing the simulation
    for k  in np.nditer(a):
        t += dt
        tic = time.time()
        q2  = np.deg2rad(k)
        for n in np.nditer(b):
            q3  = np.deg2rad(n)
            for p in np.nditer(c):
                q5  = np.deg2rad(p)
                q =np.array([0, q2, q3, np.pi, q5,0]) # Natural position as in the real case
                pin.forwardKinematics(model,data,q)
                pin.updateFramePlacements(model,data)
                OM_act = data.oMf[frameIndex] # This is the pose of the end-effector refered in the frame of the base

                pin.computeJointJacobians(model, data, q)
                J=pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED) # in the base frame
                Jbl = J[:3] # linear part of the Jacobian
                pos = np.array([OM_act.translation[0],OM_act.translation[1],OM_act.translation[2]])
                x_vec = np.array([OM_act.translation[0], 0, 0])
                y_vec = np.array([0, OM_act.translation[1], 0])
                z_vec = np.array([0, 0, OM_act.translation[2]])                 
                #Jbl = check_matrix(Jbl)
                Al = Jbl@Jbl.T

                Alx = x_vec.T@(Jbl@Jbl.T)@x_vec
                Aly = y_vec.T@(Jbl@Jbl.T)@y_vec
                Alz = z_vec.T@(Jbl@Jbl.T)@z_vec

                Alx_inv = x_vec.T@inv(Jbl@Jbl.T)@x_vec
                Aly_inv = y_vec.T@inv(Jbl@Jbl.T)@y_vec
                Alz_inv = z_vec.T@inv(Jbl@Jbl.T)@z_vec

                Al_det = (det(Al))
                Upsilon = 4*(tau_x/tau_q)*(Jbl@Jbl.T)
                if Al_det < 1e-15:  
                   Al_det = 0.0                
                if Al_det == 0.0:
                    E1 = math.nan
                else:
                    E1 = norm(Md_inv@Upsilon,ord=2) # This should be less than 1
                manip_linear = np.sqrt(Al_det)
                manipx = Alx
                manipy = Aly
                manipz = Alz

                manipx_inv = Alx_inv
                manipy_inv = Aly_inv
                manipz_inv = Alz_inv

                manip_axes = np.array([manipx,manipy,manipz])
                manip_axes_inv = np.array([manipx_inv,manipy_inv,manipz_inv])

                q_list.append(q)
                pose_list.append(pos)
                manu_list.append(manip_linear)
                E1_list.append(E1)
                manipaxes_list.append(manip_axes)
                manipaxesinv_list.append(manip_axes_inv)
                toc = time.time()
                ellapsed = toc - tic
                dt_sleep = max(0,dt - (ellapsed))
                time.sleep(dt_sleep)

    manu_list = np.array(manu_list)
    manipaxes_list = np.array(manipaxes_list)
    manipaxesinv_list = np.array(manipaxesinv_list)
    E1_list = np.array(E1_list)
    q_list = np.array(q_list)
    pose_list = np.array(pose_list)
    
    return  pose_list, q_list, manu_list, E1_list, manipaxes_list, manipaxesinv_list

In [18]:
pose, q, manip, E1, manip_axes, manip_invaxes = run_robot()

In [19]:
pd.DataFrame(pose).to_csv('pose_workspace_NE1.csv', index_label = "Index", header  = ['x','y','z'])    

In [20]:
pd.DataFrame(manip).to_csv('manip_workspace_NE1.csv', index_label = "Index", header  = ['manip'])    

In [21]:
pd.DataFrame(q).to_csv('qs_workspace_NE1.csv', index_label = "Index", header  = ['q1','q2','q3','q4','q5','q6'])    

In [22]:
pd.DataFrame(E1).to_csv('E1_workspace_NE1.csv', index_label = "Index", header  = ['E1'])    

In [26]:
pd.DataFrame(manip_axes).to_csv('manip_axes_workspace_NE1.csv', index_label = "Index", header  = ['manipx','manipy','manipz'])    

In [25]:
pd.DataFrame(manip_invaxes).to_csv('manip_invaxes_workspace_NE1.csv', index_label = "Index", header  = ['manipx','manipy','manipz'])    